In [1]:
install.packages("tidyverse")
install.packages("caTools")
install.packages("ROCR")

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
also installing the dependency ‘stringr’

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


In [2]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.4.1     ✔ dplyr   0.7.4
✔ tidyr   0.7.2     ✔ stringr 1.4.0
✔ readr   1.1.1     ✔ forcats 0.2.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [3]:
quality = read.csv("./files/quality.csv")

In [4]:
head(quality)

MemberID,InpatientDays,ERVisits,OfficeVisits,Narcotics,DaysSinceLastERVisit,Pain,TotalVisits,ProviderCount,MedicalClaims,ClaimLines,StartedOnCombination,AcuteDrugGapSmall,PoorCare
1,0,0,18,1,731,10,18,21,93,222,FALSE,0,0
2,1,1,6,1,411,0,8,27,19,115,FALSE,1,0
3,0,0,5,3,731,10,5,16,27,148,FALSE,5,0
4,0,1,19,0,158,34,20,14,59,242,FALSE,0,0
5,8,2,19,3,449,10,29,24,51,204,FALSE,0,0
6,2,0,9,2,731,6,11,40,53,156,FALSE,4,1


In [5]:
ggplot(quality) + geom_point(aes(x=OfficeVisits, y=Narcotics, color=factor(PoorCare)))

In [6]:
table(quality$PoorCare)


 0  1 
98 33 

In [7]:
quality %>% group_by(PoorCare) %>% summarise(n())

PoorCare,n()
0,98
1,33


In [8]:
library(caTools)
set.seed(88)

split = sample.split(quality$PoorCare, SplitRatio = 0.75)
summary(split)

   Mode   FALSE    TRUE 
logical      32      99 

In [9]:
qualityTrain = subset(quality, split)
qualityTest = subset(quality, !split)

In [10]:
qualityLog = glm(PoorCare ~ Narcotics + OfficeVisits, data=qualityTrain, family="binomial")

In [11]:
summary(qualityLog)


Call:
glm(formula = PoorCare ~ Narcotics + OfficeVisits, family = "binomial", 
    data = qualityTrain)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-2.06303  -0.63155  -0.50503  -0.09689   2.16686  

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept)  -2.64613    0.52357  -5.054 4.33e-07 ***
Narcotics     0.07630    0.03205   2.381  0.01728 *  
OfficeVisits  0.08212    0.03055   2.688  0.00718 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 111.888  on 98  degrees of freedom
Residual deviance:  89.127  on 96  degrees of freedom
AIC: 95.127

Number of Fisher Scoring iterations: 4


In [12]:
predictTrain = predict(qualityLog, type="response")

In [14]:
summary(predictTrain)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
0.06623 0.11912 0.15967 0.25253 0.26765 0.98456 

In [17]:
tapply(predictTrain, qualityTrain$PoorCare, mean)

0         1 
0.1894512 0.4392246

In [22]:
boxplot(predictTrain~qualityTrain$PoorCare, horizontal = T)

In [23]:
table(qualityTrain$PoorCare, predictTrain > 0.5)

   
    FALSE TRUE
  0    70    4
  1    15   10

In [34]:
sens = 10/(10+15)
spec = 70/(70+4)
acc = (70+10)/99
print(sens)
print(spec)
print(acc)

[1] 0.4
[1] 0.9459459
[1] 0.8080808


In [29]:
table(qualityTrain$PoorCare, predictTrain > 0.7)

   
    FALSE TRUE
  0    73    1
  1    17    8

In [35]:
sens = 8/25
spec = 73/74
acc = (73+8)/99
print(sens)
print(spec)
print(acc)

[1] 0.32
[1] 0.9864865
[1] 0.8181818


In [31]:
table(qualityTrain$PoorCare, predictTrain > 0.2)

   
    FALSE TRUE
  0    54   20
  1     9   16

In [36]:
sens = 16/25
spec = 54/74
acc = (54+16)/99
print(sens)
print(spec)
print(acc)

[1] 0.64
[1] 0.7297297
[1] 0.7070707


In [51]:
library(ROCR)
ROCRpred = prediction(predictTrain,qualityTrain$PoorCare)
ROCRperf = performance(ROCRpred, "tpr", "fpr")
plot(ROCRperf, colorize=T)

In [54]:
plot(ROCRperf, colorize=T, print.cutoffs.at=seq(0, 1, 0.1), text.adj=c(-0.5, 0.5))

In [55]:
predictTest = predict(qualityLog, type="response", newdata=qualityTest)

In [57]:
table(qualityTest$PoorCare, predictTest > 0.5)

   
    FALSE TRUE
  0    23    1
  1     5    3

In [58]:
ROCRpred = prediction(predictTest,qualityTest$PoorCare)
ROCRperf = performance(ROCRpred, "tpr", "fpr")
plot(ROCRperf, colorize=T)